# Setup Perfmon Data Collection

This notebook contains instructions on how to setup perfmon data collection on each server to be monitored, and import the data into Central Monitoring Server.

We create 2 Grafana dashboards on top of collected data using Central Server.

## Instructions -

1. Part 01 - Setup Baselining of SqlServer using Perfmon & Powershell
2. Part 02 - Configure Grafana for Visualization on baselined data

  

Initiatize **$Servers** with list of Server Names where Perfmon Baselining has to be done.

In [ ]:
$Servers = @('localhost')

# Part 01 - Setup Baselining of SqlServer using Perfmon & PowerShell

## 1\. Create required objects in \[DBA\] database using below script. Make sure to change the data & log files path.

If a similar DBA database exists where data collection should be performed, then this step can be skipped.

In [ ]:
$sqlCreateDatabase = @"
USE master
GO
create database [DBA]
--on (name = 'DBA', filename = 'E:\MSSQL14.V17\Data\DBA.mdf' , SIZE = 500MB , FILEGROWTH = 200MB )
--log on (name = 'DBA_log', filename = 'E:\MSSQL14.V17\Log\DBA_log.ldf' , SIZE = 500MB , FILEGROWTH = 200MB );
GO
"@

Invoke-DbaQuery -SqlInstance $Servers -Query $sqlCreateDatabase;

$sqlCreateObjects = @"
CREATE SCHEMA [bkp]
GO
CREATE SCHEMA [poc]
GO
CREATE SCHEMA [stg]
GO
CREATE SCHEMA [tst]
GO

create table dbo.perfmon_files
(	server_name varchar(100) not null, 
	file_name varchar(255) not null, 
	file_path varchar(255) not null, 
	collection_time_utc datetime2 not null default SYSUTCDATETIME()
	,constraint pk_perfmon_files primary key (file_name, collection_time_utc)
)
go

create partition function pf_dba (datetime2)
as range right for values ('2022-03-25 00:00:00.0000000')
go

create partition scheme ps_dba as partition pf_dba all to ([primary])
go

-- drop table [dbo].[performance_counters]
create table [dbo].[performance_counters]
(
	[collection_time_utc] [datetime2](7) NOT NULL,
	[computer_name] [varchar](200) NOT NULL,
	[path] [nvarchar](2000) NOT NULL,
	[object] [varchar](255) NOT NULL,
	[counter] [varchar](255) NOT NULL,
	[value] numeric(38,10) NULL,
	[instance] [nvarchar](255) NULL
) on ps_dba ([collection_time_utc])
go

create clustered index ci_performance_counters on [dbo].[performance_counters] ([collection_time_utc], object, counter, [instance], [value])
go


/* Validate Partition Data */
SELECT SCHEMA_NAME(o.schema_id)+'.'+ o.name as TableName,
	pf.name as PartitionFunction,
	ds.name AS PartitionScheme, 
	p.partition_number AS PartitionNumber, 
	CASE pf.boundary_value_on_right WHEN 1 THEN 'RIGHT' ELSE 'LEFT' END AS PartitionFunctionRange, 
	prv_left.value AS LowerBoundaryValue, 
	prv_right.value AS UpperBoundaryValue, 
	fg.name AS FileGroupName,
	p.[row_count] as TotalRows,
	CONVERT(DECIMAL(12,2), p.reserved_page_count*8/1024.0) as ReservedSpaceMB,
	CONVERT(DECIMAL(12,2), p.used_page_count*8/1024.0) as UsedSpaceMB
FROM sys.dm_db_partition_stats AS p (NOLOCK)
	INNER JOIN sys.indexes AS i (NOLOCK) ON i.[object_id] = p.[object_id] AND i.index_id = p.index_id
	INNER JOIN sys.data_spaces AS ds (NOLOCK) ON ds.data_space_id = i.data_space_id
	INNER JOIN sys.objects AS o (NOLOCK) ON o.object_id = p.object_id
	INNER JOIN sys.partition_schemes AS ps (NOLOCK) ON ps.data_space_id = ds.data_space_id
	INNER JOIN sys.partition_functions AS pf (NOLOCK) ON pf.function_id = ps.function_id
	INNER JOIN sys.destination_data_spaces AS dds2 (NOLOCK) ON dds2.partition_scheme_id = ps.data_space_id AND dds2.destination_id = p.partition_number
	INNER JOIN sys.filegroups AS fg (NOLOCK) ON fg.data_space_id = dds2.data_space_id
	LEFT OUTER JOIN sys.partition_range_values AS prv_left (NOLOCK) ON ps.function_id = prv_left.function_id AND prv_left.boundary_id = p.partition_number - 1
	LEFT OUTER JOIN sys.partition_range_values AS prv_right (NOLOCK) ON ps.function_id = prv_right.function_id AND prv_right.boundary_id = p.partition_number
WHERE
	OBJECTPROPERTY(p.[object_id], 'IsMSShipped') = 0
ORDER BY p.partition_number;	
go

/* Add boundaries to partition. 1 boundary per hour */
set nocount on;
declare @partition_boundary datetime2;
declare @target_boundary_value datetime2; /* 3 months back date */
set @target_boundary_value = DATEADD(mm,DATEDIFF(mm,0,GETDATE())-3,0);
set @target_boundary_value = '2022-03-25 19:00:00.000'

declare cur_boundaries cursor local fast_forward for
		select convert(datetime2,prv.value) as boundary_value
		from sys.partition_range_values prv
		join sys.partition_functions pf on pf.function_id = prv.function_id
		where pf.name = 'pf_dba' and convert(datetime2,prv.value) < @target_boundary_value
		order by prv.value asc;

open cur_boundaries;
fetch next from cur_boundaries into @partition_boundary;
while @@FETCH_STATUS = 0
begin
	--print @partition_boundary
	alter partition function pf_dba() merge range (@partition_boundary);

	fetch next from cur_boundaries into @partition_boundary;
end
CLOSE cur_boundaries
DEALLOCATE cur_boundaries;
go


/* Remove boundaries with retention of 3 months */
set nocount on;
declare @current_boundary_value datetime2;
declare @target_boundary_value datetime2; /* last day of new quarter */
set @target_boundary_value = DATEADD (dd, -1, DATEADD(qq, DATEDIFF(qq, 0, GETDATE()) +2, 0));

select top 1 @current_boundary_value = convert(datetime2,prv.value)
from sys.partition_range_values prv
join sys.partition_functions pf on pf.function_id = prv.function_id
where pf.name = 'pf_dba'
order by prv.value desc;

select [@current_boundary_value] = @current_boundary_value, [@target_boundary_value] = @target_boundary_value;

while (@current_boundary_value < @target_boundary_value)
begin
	set @current_boundary_value = DATEADD(hour,1,@current_boundary_value);
	--print @current_boundary_value
	alter partition scheme ps_dba next used [primary];
	alter partition function pf_dba() split range (@current_boundary_value);	
end
go

"@

Invoke-DbaQuery -SqlInstance $Servers -Database 'DBA' -Query $sqlCreateObjects



## 2\. Start Perfmon data collection using template **NonSql-Files\\DBA\_PerfMon\_NonSQL\_Collector\_Template.xml**

Copy the template file **NonSql-Files\\DBA\_PerfMon\_NonSQL\_Collector\_Template.xml** on path where perfmon collector files would be saved. This directory should be at least 4 gb of size. 

Then in below script, update **$data\_collector\_template\_path** variable value on line 3. Now execute below script. This should create a perfmon data collector set named \[DBA\].

In [26]:
# Copy paste the template file on folder that would contain Perfmon data collection logs
    # Point the template path
$data_collector_template_path = "E:\Perfmon\DBA_PerfMon_All_Counters_Template.xml";
$data_collector_set_name = 'DBA';

# find Perfmon data collection logs folder path
$collector_root_directory = Split-Path $data_collector_template_path -Parent
$log_file_path = "$collector_root_directory\$data_collector_set_name"
$file_rotation_time = '00:30:00'
$sample_interval = '00:00:10'

logman import -name "$data_collector_set_name" -xml "$data_collector_template_path"
logman update -name "$data_collector_set_name" -f bin -cnf "$file_rotation_time" -o "$log_file_path" -si "$sample_interval"
logman start -name "$data_collector_set_name"

<#
logman stop -name "$data_collector_set_name"
logman delete -name "$data_collector_set_name"
#>


The command completed successfully.


The command completed successfully.


The command completed successfully.


## 3\. Create jobs \[**(dba) Collect-PerfmonData**\], \[**(dba) Purge-DbaMetrics - Daily**\] & \[**(dba) Partitions-Maintenance**\]

Execute below tsql

In [11]:
Get-Location
$log_file_path
$repoFolder
$collector_root_directory


Path              
----              
D:\GitHub-Personal
D:\GitHub-Personal\SqlServer-Baselining-Grafana\Perfmon
D:\GitHub-Personal\SqlServer-Baselining-Grafana
D:\GitHub-Personal\SqlServer-Baselining-Grafana\Perfmon


